## No Lambdas

Example: 1 in the second to last place

<img src = "../Week3_NFAs/1in2splace.png">

In [1]:
def eval_nfa(s, q, delta, F, verbose=False):
    """
    Evaluate a standard NFA with no lambda arrows
    
    Parameters
    ----------
    s: string
        Input to try
    q: string
        Start state
    delta: (string, string) -> set([string])
        Transition function
    F: set([string])
        Accept states
    verbose: bool
        If True, print info about sequence of states that are visited
    """
    states = set([q])
    for c in s:
        if verbose:
            print(states)
        next_states = set([])
        for p in states:
            if (p, c) in delta:
                next_states = next_states.union(delta[(p, c)])
        states = next_states
    if verbose:
        print(states)
    return len(states.intersection(F)) > 0


delta = {
    ("q", "0"):{"q"},
    ("q", "1"):{"q", "q2"},
    ("q2", "0"):{"q0"},
    ("q2", "1"):{"q0"}
}

F = {"q0"}
q = "q"
s = "11010"
print(s, "Accepts:", eval_nfa(s, q, delta, F, verbose=True))

{'q'}
{'q2', 'q'}
{'q2', 'q', 'q0'}
{'q', 'q0'}
{'q2', 'q'}
{'q', 'q0'}
11010 Accepts: True


This matches the picture we drew

<img src = "../Week3_NFAs/1In2sPlaceInput.svg" width="25%">

## With Lambdas

Let's try the union example

<img src = "../Week3_NFAs/UnionExample.png">

We'll use python None to represent $\lambda$

In [2]:
def expand_lambdas_rec(delta, s, states):
    """
    Helper method for recursive lambda arrow following
    """
    if (s, None) in delta:
        states.update(delta[(s, None)]) # Add all elements coming out of lambda to the set
        for p in delta[(s, None)]:
            # Continue exploring other lambda arrows coming out of these states
            expand_lambdas_rec(delta, p, states)

def expand_lambdas(delta, states):
    """
    Recursively follow all arrows with lambdas and add
    whatever states are visited to the set of states
    """
    orig_states = list(states)
    for s in orig_states:
        expand_lambdas_rec(delta, s, states)

def eval_nfa_withlambda(s, q, delta, F, verbose=False):
    """
    Evaluate an NFA that includes lambda arrows
    
    Parameters
    ----------
    s: string
        Input to try
    q: string
        Start state
    delta: (string, string) -> set([string])
        Transition function.  Lambda transitions are represented as (string, None)
    F: set([string])
        Accept states
    verbose: bool
        If True, print info about sequence of states that are visited
    """
    states = set([q])
    for c in s:
        if verbose:
            print(states)
        expand_lambdas(delta, states)
        if verbose:
            print("After lambda expansion:", states)
        next_states = set([])
        for p in states:
            if (p, c) in delta:
                next_states = next_states.union(delta[(p, c)])
        states = next_states
    if verbose:
        print(states)
    expand_lambdas(delta, states)
    if verbose:
        print("After lambda expansion:", states)
    return len(states.intersection(F)) > 0



delta = {
    ("q", None):{"q0", "peven"},
    
    ("q0", "0"):{"q0"},
    ("q0", "1"):{"q1"},
    ("q1", "0"):{"q1"},
    ("q1", "1"):{"q2"},
    ("q2", "0"):{"q2"},
    ("q2", "1"):{"q2"},
    
    ("peven", "0"):{"podd"},
    ("peven", "1"):{"peven"},
    ("podd", "0"):{"peven"},
    ("podd", "1"):{"podd"}
}
q = "q"
F = {"q0", "q1", "podd"}
s = "00011"
print(s, "Accepts:", eval_nfa_withlambda(s, q, delta, F, verbose=True))

{'q'}
After lambda expansion: {'q', 'q0', 'peven'}
{'q0', 'podd'}
After lambda expansion: {'q0', 'podd'}
{'q0', 'peven'}
After lambda expansion: {'q0', 'peven'}
{'q0', 'podd'}
After lambda expansion: {'q0', 'podd'}
{'q1', 'podd'}
After lambda expansion: {'q1', 'podd'}
{'q2', 'podd'}
After lambda expansion: {'q2', 'podd'}
00011 Accepts: True


This matches the picture we drew

<img src = "../Week3_NFAs/UnionExampleInput.svg" width="25%">

## More Complicated Example with Lambdas

<img src = "../Week3_NFAs/LambdaEx.png">

In [3]:
delta = {
    ("q0", None):{"q1"},
    ("q0", "0"):{"q0"},
    ("q0", "1"):{"q1"},
    
    ("q1", "1"):{"q1"},
    ("q1", None):{"q2"},
    
    ("q2", "0"):{"q2"}
}
q = "q0"
F = {"q2"}
s = "0011"
print(s, "Accepts:", eval_nfa_withlambda(s, q, delta, F, verbose=True))

{'q0'}
After lambda expansion: {'q2', 'q1', 'q0'}
{'q2', 'q0'}
After lambda expansion: {'q2', 'q1', 'q0'}
{'q2', 'q0'}
After lambda expansion: {'q2', 'q1', 'q0'}
{'q1'}
After lambda expansion: {'q2', 'q1'}
{'q1'}
After lambda expansion: {'q2', 'q1'}
0011 Accepts: True


Notice that this matches our picture from before:

<img src = "../Week3_NFAs/LambdaEx.svg" width="35%">

But one crucial difference is if two clones are at the same state at any moment in time, <b>we condense them into a single copy of that state</b> due to our use of the set object.  This is a huge advantage of coding up NFAs in practice, and it prevents an exponential blowup of clones.  In particular, we can never have more than <b>|Q|</b> states in memory at one time